# Classifier Tests Jupyter Notebook

In [ ]:
import importlib

# Import myutils.py from mysklearn forlder
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# Import mypytable.py from mysklearn folder
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# Import myclassifiers.py from mysklearn folder
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyRandomForestClassifier, MyKNeighborsClassifier, MyDecisionTreeClassifier

# Import myevaluation.py from mysklearn folder
import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

## Random Forest Classifier

In [ ]:
# Code goes here

## kNN Classifier

In [ ]:
# Code goes here

## Decision Tree Classifier

In [ ]:
# Code goes here